<a href="https://colab.research.google.com/github/luisosmx/Python_exercises/blob/main/Cloud_storage_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Download Cloud Storage libraries


In [ ]:
!pip install google-cloud-storage
!pip install pandas google-cloud-storage


#Download files from Cloud Storage

In [ ]:
from google.cloud import storage
import os
import pandas as pd

# Configura la variable de entorno para las credenciales
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path/file.json'

# Crea un cliente de almacenamiento
storage_client = storage.Client()

# Nombre del bucket y objeto que deseas leer
bucket_name = 'bucketdata_storage'
object_name = '/content/inah_visitantes_zonas_general_anual_2023.csv'
destination = '/content/file.csv'
# Obtiene el objeto
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(object_name)
#Descarga el contenido del objeto como bytes
blob.download_to_filename(destination)
print("Archivo descargado como 'file.csv")

##Clean files

In [ ]:

# making dataframe
df = pd.read_csv(destination, encoding='latin-1')
column_names = df.columns.tolist()

# Lista de nombres de todas las columnas
all_column_names = ['Estado', 'Clave SIINAH', 'Tipo', 'Centro INAH', 'enero', 'Unnamed: 5', 'febrero', 'Unnamed: 7', 'marzo', 'Unnamed: 9', 'abril', 'Unnamed: 11', 'mayo', 'Unnamed: 13', 'junio', 'Unnamed: 15', 'julio', 'Unnamed: 17']

# Nombres de las columnas que deseas mantener
filter_column = ['estado', 'tipo', 'centro_inah', 'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio']

# Filtra los nombres de las columnas que coinciden con los que deseas mantener
filtered_column_names = [col for col in all_column_names if col in filter_column]

filtered_df = df.loc[:, filtered_column_names]
new_column_names = ['estado', 'tipo', 'centro_inah', 'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio']
df.columns = new_column_names


In [ ]:
df_types = df.dtypes
print(df_types)

estado         object
tipo           object
centro_inah    object
enero          object
febrero        object
marzo          object
abril          object
mayo           object
junio          object
julio          object
dtype: object


In [ ]:
dframe = df_types.astype(str)
print(dframe)

estado         object
tipo           object
centro_inah    object
enero          object
febrero        object
marzo          object
abril          object
mayo           object
junio          object
julio          object
dtype: object


In [ ]:
from google.cloud import bigquery
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path/file.json'

client = bigquery.Client()
table_id = "data-project-395314.data_project.inah_visitantes"

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField("estado", "STRING"),
        bigquery.SchemaField("tipo", "STRING"),
        bigquery.SchemaField("centro_inah", "STRING"),
        bigquery.SchemaField("enero", "STRING"),
        bigquery.SchemaField("febrero", "STRING"),
        bigquery.SchemaField("marzo", "STRING"),
        bigquery.SchemaField("abril", "STRING"),
        bigquery.SchemaField("mayo", "STRING"),
        bigquery.SchemaField("junio", "STRING"),
        bigquery.SchemaField("julio", "STRING"),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)


Loaded 177 rows and 10 columns to data-project-395314.data_project.inah_visitantes
